## 🚩 필요한 Libray Import

In [ ]:
import numpy as np
import os
import random
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import seaborn as sns

from PIL import Image
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold

from tensorflow.keras.preprocessing.image import load_img,img_to_array

## 🚩 Train Data Load

In [ ]:
train_data_path = '../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv'
train_data = pd.read_csv(train_data_path)

train_data

## 🚩 각 class 별 Data 개수 확인 & Data 양 분포 확인

In [ ]:
pd.value_counts(train_data['cultivar'])

In [ ]:
plt.figure(figsize=[24, 6], dpi=200)
sns.countplot(x=train_data['cultivar'])
plt.xticks(rotation=60)
plt.show()

## 🚩 CLAHE
----
* **Color Image 기준 -> R,G,B**
* **CLAHE란, 이미지를 일정한 크기를 가진 작은 블록으로 구분하고, 블록별로 히스토그램 균일화를 시행하여 이미지 전체에 적용**
* **cv.createCLAHE() 함수를 적용하여 Limit를 적용하여 히스토그램 균일화 진행**

### ➿ 원본 이미지 히스토 그램 확인

In [ ]:
dir = '../input/sorghum-cultivar-identification-512512/train'

for fname in train_data['image'][:3]:
    image  = load_img(os.path.join(dir, fname))
    grayscale = load_img(os.path.join(dir, fname),color_mode = "grayscale")
    gray_array=  img_to_array(grayscale)
    image_array = img_to_array(image)

    plt.imshow(image)
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=1,ncols=4, sharey=True, figsize=(24,5))

    ax1.hist(image_array[:,:,0].ravel(),256,[0,256],color='red')
    plt.ylim(0,20000)
    ax2.hist(image_array[:,:,1].ravel(),256,[0,256], color='green')
    ax3.hist(image_array[:,:,1].ravel(),256,[0,256], color='blue')
    ax4.hist(gray_array.ravel(),256,[0,256])
    plt.show()

### ➿ 원본 이미지에 CLAHE 적용

In [ ]:
import cv2 as cv
from PIL import Image as Img

for fname in train_data['image'][:3]:
    image = load_img(os.path.join(dir, fname))
    clahe = cv.createCLAHE(clipLimit=40, tileGridSize=(10,10))
    t = np.asarray(image)
    t = cv.cvtColor(t, cv.COLOR_BGR2HSV)
    t[:,:,-1] = clahe.apply(t[:,:,-1])
    t = cv.cvtColor(t, cv.COLOR_HSV2BGR)
    t = Img.fromarray(t)

    plt.imshow(t)
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=1,ncols=4, sharey=True, figsize=(24,5))

    ax1.hist(image_array[:,:,0].ravel(),256,[0,256],color='red')
    plt.ylim(0,20000)
    ax2.hist(image_array[:,:,1].ravel(),256,[0,256], color='green')
    ax3.hist(image_array[:,:,1].ravel(),256,[0,256], color='blue')
    ax4.hist(gray_array.ravel(),256,[0,256])
    plt.show()

## 🚩 이미지를 원하는 크기로 자르거나 늘림
----
* **zoom 수행 -> 입력 : 이미지, 이미지 변형을 원하는 범위**
* **이미지 확대, 축소 2가지 이미지 변형 수행**
* **.resize_with_crop_or_pad() 함수 사용하여 target 이미지 크기를 0으로 채움**

In [ ]:
def random_zoom(image, zoom_range):
    # 난수 생성
    factor = np.random.uniform(low =- zoom_range, high = zoom_range)
    
    # int형이 아닌 경우 image를 array로 변형
    if not isinstance(image, np.ndarray):
        arr = tf.keras.utils.img_to_array(image)

        tf.image.resize_with_crop_or_pad(tf.image.resize(arr,
                                                         (int(arr.shape[0] + arr.shape[0] * factor),
                                                          int(arr.shape[1] + arr.shape[1] * factor))),
                                         target_height=512,
                                         target_width=512)

    return tf.image.resize_with_crop_or_pad(tf.image.resize(image,
                                                            (int(image.shape[0] + image.shape[0] * factor),
                                                             int(image.shape[1] + image.shape[1] * factor))),
                                            target_height=512,
                                            target_width=512)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=[12, 6], dpi=300)

for i in range(3):
    zoomed_img = random_zoom(tf.keras.utils.img_to_array(Image.open(os.path.join(dir, train_data['image'][0]))), 0.5)
    axes[i].imshow(tf.keras.utils.array_to_img(zoomed_img))

plt.show()

## 🚩 이미지를 원하는 각도 만큼 회전
----
* **rotation 수행 -> 입력 : 이미지, 원하는 회전 각도 범위**
* **.image.rotate() 함수 사용하여 radian 단위로 시계 반대 방향으로 회전**
* **.fill_mode='nearest'를 이용하여 경계는 가장 가까운 픽셀만큼 확장**

In [ ]:
def random_rotate(image, rotation_range):
    factor = np.random.uniform(low = -rotation_range, high = rotation_range)

    if not isinstance(image, np.ndarray):
        arr = tf.keras.utils.img_to_array(image)

        return tfa.image.rotate(arr, angles=factor, fill_mode='nearest')

    else:
        return tfa.image.rotate(image, angles=factor, fill_mode='nearest')


In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=[12, 6], dpi=300)

for i in range(3):
    rotated_img = random_rotate(Image.open(os.path.join(dir, train_data['image'][0])), 20)
    axes[i].imshow(tf.keras.utils.array_to_img(rotated_img))

plt.show()

## 🚩 zoom + rotaion

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=[12, 6], dpi=300)

for i in range(3):
    zoomed_img = random_zoom(tf.keras.utils.img_to_array(Image.open(os.path.join(dir, train_data['image'][0]))), 0.5)
    rotated_img = random_rotate(zoomed_img, 20)
    axes[i].imshow(tf.keras.utils.array_to_img(rotated_img))

plt.show()

## 🚩 Image Crop
----
* **.image.random_crop() 함수 사용하여 주어진 크기로 텐서(이미지)를 무작위로 자르기**
* **무작위로 자르기 때문에 자르는 위치를 알지 못함**
* **자르고자 하는 이미지 크기 : 512 -> 128**
* **Color Image 기준**

In [ ]:
# color image이기 때문에 자르고자 하는 이미지가 총 3개
central_crop_width = (0.35, 0.65, 0.75)
central_crop_height = (0.35, 0.9, 0.75)

for w_factor, h_factor in zip(central_crop_width, central_crop_height):
    h, w = tf.keras.utils.img_to_array(Image.open(os.path.join(dir, train_data['image'][0]))).shape[:2]
    print(int(w * w_factor), int(h * h_factor))

In [ ]:
def resize_to_smaller_side(img,small_side_to=224, crop_window=(128, 128, 3), copies=3):
    h, w = img.shape[:2]
    crops = []

    if h < w:
        resized = tf.image.resize(img, size=(small_side_to, w))
        
    elif w < h:
        resized = tf.image.resize(img, size=(h, small_side_to))
        
    else:
        resized = tf.image.resize(img, (small_side_to, w))

    # color 이미지이기 때문에 3번 반복
    for _ in range(copies):
        crops.append(tf.image.random_crop(resized, crop_window))

    return crops

In [ ]:
img = tf.keras.utils.img_to_array(Image.open(os.path.join(dir, train_data['image'][0])))
crops = resize_to_smaller_side(tf.image.resize(img, size=(512, 512)))

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=[12, 6], dpi=300)

for i, crop in enumerate(crops):
    axes[i].imshow(tf.keras.utils.array_to_img(crop))

plt.show()

## 🚩 Image Zoom + Rotation + Crop
----
* **.image.crop_to_bounding_box()를 사용하여 image를 지정된 범위의 bounding box로 자름**
* **zoom, rotation, crop 3가지 기능 수행 가능**
* **Color Image 기준**

In [ ]:
def cropping(filename, dir,
             rotate_range=10,
             flipping=True,
             zoom_range=0.5,
             central_crop_width=(0.35, 0.65, 0.75),central_crop_height=(0.35, 0.9, 0.45),
             random_crop_window=(128, 128, 3),
             small_side_to=224,
             copies=3):
    
    arr = tf.keras.utils.img_to_array(Image.open(os.path.join(dir, filename)))
    crops = []

    if isinstance(central_crop_width, (list, tuple, np.ndarray)):

        for w_factor, h_factor in zip(central_crop_width, central_crop_height):
            h, w = arr.shape[:2]
            offset_h = (h - h_factor * h) // 2
            offset_w = (w - w_factor * w) // 2

            crops.append(
                tf.image.crop_to_bounding_box(arr, int(offset_h), int(offset_w), int(h * h_factor), int(w * w_factor))
            )

    return crops

In [ ]:
dir = '../input/sorghum-cultivar-identification-512512/train'
filename = train_data['image'][0]

crops = cropping(filename, dir)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=[12, 10], dpi=300)
axes = axes.ravel()

for i, crop in enumerate(crops):
    axes[i].imshow(tf.keras.utils.array_to_img(crop))

plt.show()